In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D,MaxPooling2D,Flatten
import pandas as pd


In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_11d6882aba4a4088b92e5b1911d45304 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='JEeVb9CNqul_jiLx5SB0D_n-VhY6aFlZ1P2OCgzWlHVq',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_11d6882aba4a4088b92e5b1911d45304.get_object(Bucket='digitalnaturalistdeployment-donotdelete-pr-ufh1rcmi5nugwp', Key='New.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
pwd

'/home/wsuser/work'

In [9]:
import os
filenames=os.listdir('/home/wsuser/work/New')

In [10]:
from tensorflow.keras.preprocessing .image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
x_train=train_datagen.flow_from_directory(r"/home/wsuser/work/New/traindata",target_size = (64,64),batch_size = 32,class_mode = "categorical")
x_test=test_datagen.flow_from_directory(r"/home/wsuser/work/New/testdata",target_size = (64,64),batch_size = 32,class_mode = "categorical")

Found 8305 images belonging to 9 classes.
Found 900 images belonging to 9 classes.


In [12]:
x_train.class_indices

{'ANTBIRD': 0,
 'PEACOCK': 1,
 'WILD TURKEY': 2,
 'gatto': 3,
 'mucca': 4,
 'pecora': 5,
 'rose': 6,
 'sunflower': 7,
 'tulip': 8}

In [13]:
model=Sequential()

In [14]:
model.add(Convolution2D(32,(3,3),input_shape = (64,64,3)))

In [15]:
model.add(MaxPooling2D((2,2)))

In [16]:
model.add(Flatten())

In [17]:
model.add(Dense(units = 128, kernel_initializer= "random_uniform",activation = "relu"))

In [18]:
model.add(Dense(units = 9, kernel_initializer= "random_uniform",activation = "softmax"))

In [19]:
model.compile(optimizer= "rmsprop",loss = "categorical_crossentropy" , metrics =["accuracy"])

In [20]:
model.fit_generator(x_train,steps_per_epoch =251 ,epochs = 10, validation_data = x_test,validation_steps =28 )

/tmp/wsuser/ipykernel_227/3144264315.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch =251 ,epochs = 10, validation_data = x_test,validation_steps =28 )


Epoch 1/10
251/251 [==============================] - 77s 304ms/step - loss: 1.4655 - accuracy: 0.4830 - val_loss: 1.7942 - val_accuracy: 0.4643
Epoch 2/10
251/251 [==============================] - 81s 321ms/step - loss: 1.1774 - accuracy: 0.5719 - val_loss: 1.3458 - val_accuracy: 0.5614
Epoch 3/10
251/251 [==============================] - 76s 302ms/step - loss: 1.0773 - accuracy: 0.6075 - val_loss: 1.2842 - val_accuracy: 0.5837- loss:
Epoch 4/10
251/251 [==============================] - 81s 324ms/step - loss: 1.0131 - accuracy: 0.6259 - val_loss: 1.0689 - val_accuracy: 0.6685
Epoch 5/10
251/251 [==============================] - 74s 296ms/step - loss: 0.9516 - accuracy: 0.6517 - val_loss: 1.0041 - val_accuracy: 0.6775
Epoch 6/10
251/251 [==============================] - 72s 288ms/step - loss: 0.8978 - accuracy: 0.6734 - val_loss: 1.1526 - val_accuracy: 0.6518: 3s - loss: 0.9017 - accuracy - ETA: 2s - loss: 0
Epoch 7/10
251/251 [==============================] - 68s 269ms/step - lo

In [21]:
model.save("natur1.h5")

In [22]:
!tar -zcvf digital_naturalist_model.tgz natur1.h5

natur1.h5


In [23]:
ls -1

digital_naturalist_model.tgz
natur1.h5
New/


In [24]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 21.8 MB/s eta 0:00:01


In [46]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"eYrTB0mZJUdt3xLP1sNLIr02vKWw7ZRLSQblPBOqrq8_"
}
client = APIClient(wml_credentials)

In [47]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [48]:
space_uid=guid_from_space_name(client,'digi_naturalist_deploy')
print("Space UID="+space_uid)

Space UID=60295b97-1414-4e2e-8f18-dbb9c405edeb


In [49]:
client.set.default_space(space_uid)

'SUCCESS'

In [50]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a17-24d8-5082-900f-0ab31fbfd3cb  base
scikit-learn_0.22-py3.6        154010f

In [52]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9-horovod")
software_spec_uid

'dda170cc-ca67-5da7-9b7a-cf84c6987fae'

In [ ]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_id(model_details)

Failure during getting trained models details. (GET https://us-south.ml.cloud.ibm.com/ml/v4/trainings/image-classification-model_new.tgz?version=2021-06-24&space_id=60295b97-1414-4e2e-8f18-dbb9c405edeb)
Status code: 404, body: {"trace":"89ad5a00c9af98d1ff83c204471ab06a","errors":[{"code":"training_job_run_not_found","message":"Backend persistence error (404): GET request failed","more_info":"http://watson-ml-api.mybluemix.net/"}],"status_code":"404"}
Unexpected type of 'model parameter', expected: model path / training_id, actual: 'image-classification-model_new.tgz'.


UnexpectedType: Unexpected type of 'model parameter', expected: model path / training_id, actual: 'image-classification-model_new.tgz'.